In [1]:
import pandas as pd
import numpy as np

In [2]:
file = 'uber_data.csv'

df = pd.read_csv(file)

df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-03-01 00:00:00,2016-03-01 00:07:55,1,2.50,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,2016-03-01 00:00:00,2016-03-01 00:11:06,1,2.90,-73.983482,40.767925,1,N,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2016-03-01 00:00:00,2016-03-01 00:31:06,2,19.98,-73.782021,40.644810,1,N,-73.974541,40.675770,1,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,2,2016-03-01 00:00:00,2016-03-01 00:00:00,3,10.78,-73.863419,40.769814,1,N,-73.969650,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,2,2016-03-01 00:00:00,2016-03-01 00:00:00,5,30.43,-73.971741,40.792183,3,N,-74.177170,40.695053,1,98.0,0.0,0.0,0.00,15.50,0.3,113.80


In [3]:
df = df.drop_duplicates().reset_index(drop=True)
df['trip_id'] = df.index

DIMENSION TABLES 
-----------------

1. Datetime 
    Pick-up and drop-off Datetime
2. Passenger count
3. Pick-up location
4. Drop-off location 
5. Rate-code
6. Payment type
7. Trip distance
8. Tax surcharges

We didn't create dim on fare as they can be very different from each one and depends on multiple factors.
So, we also realised that columns which are very less dependent on each other and can have multiple similar occurance can be put into dim.

In [112]:
# Datetime 

# Convert date-time cols to proper format
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

datetime_cols = ['tpep_pickup_datetime', 'tpep_dropoff_datetime']
datetime_dim = df[datetime_cols][:]

datetime_dim['pickup_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pickup_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pickup_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pickup_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pickup_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday


datetime_dim['datetime_id'] = datetime_dim.index+1

datetime_dim = datetime_dim[['datetime_id', 
                             'tpep_pickup_datetime', 'pickup_hour', 'pickup_day', 'pickup_month', 'pickup_year', 'pickup_weekday',
                              'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]


# Passenger_count

passenger_count_dim = df[['passenger_count']].drop_duplicates().reset_index(drop=True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index+1


# Pick-up location

pickup_location_dim = df[['pickup_latitude', 'pickup_longitude']].drop_duplicates().reset_index(drop=True)
pickup_location_dim['pickup_location_id'] = pickup_location_dim.index+1


# Drop-off location

dropoff_location_dim = df[['dropoff_latitude', 'dropoff_longitude']].drop_duplicates().reset_index(drop=True)
dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index+1


# Rate-code

rate_code_dict = {
    1: 'Standard rate',
    2: 'JFK',
    3: 'Newark',
    4: 'Nassau or Westchester',
    5: 'Negotiated fare',
    6: 'Group ride',
}
rate_code_dim = df[['RatecodeID']].drop_duplicates().reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index+1
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_dict)
rate_code_dim = rate_code_dim[['rate_code_id', 'RatecodeID', 'rate_code_name']]


# Payement type
payment_type_dict = {
    1: "Credit card",
    2: "Cash",
    3: "No charge",
    4: "Dispute",
    5: "Unknown", 
    6: "Voided trip"
}
payment_type_dim = df['payment_type'].drop_duplicates().reset_index(drop=True)
payment_type_dim = payment_type_dim.to_frame()
payment_type_dim['payment_type_id'] = payment_type_dim.index+1
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_dict)
payment_type_dim = payment_type_dim[['payment_type_id', 'payment_type', 'payment_type_name']]


# Trip distance 

trip_distance_dim = df['trip_distance'].drop_duplicates().reset_index(drop=True)
trip_distance_dim = trip_distance_dim.to_frame()
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index+1
trip_distance_dim['trip_distance'] = pd.to_numeric(trip_distance_dim['trip_distance'])


# Tax surcharges
tax_dim = df[['extra', 'mta_tax', 'tolls_amount', 'improvement_surcharge']].drop_duplicates().reset_index(drop=True)
tax_dim['tax_surcharge_id'] = tax_dim.index+1

In [113]:
# Fact tables

fact_table = df.merge(datetime_dim, on=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])\
                .merge(passenger_count_dim, on='passenger_count')\
                .merge(pickup_location_dim, on=['pickup_latitude', 'pickup_longitude'])\
                .merge(dropoff_location_dim, on=['dropoff_latitude', 'dropoff_longitude'])\
                .merge(rate_code_dim, on='RatecodeID')\
                .merge(payment_type_dim, on='payment_type')\
                .merge(trip_distance_dim, on='trip_distance')\
                .merge(tax_dim, on=['extra', 'mta_tax', 'tolls_amount', 'improvement_surcharge'])\
                [['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
                    'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id',
                    'payment_type_id', 'fare_amount', 'tip_amount', 'tax_surcharge_id', 'total_amount']]

In [114]:
fact_table.head()

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type_id,fare_amount,tip_amount,tax_surcharge_id,total_amount
0,0,1,1,1,1,1,N,1,1,1,9.0,2.05,1,12.35
1,87209,1,87210,1,1,1,N,117,7,1,9.0,1.00,1,11.30
2,90628,1,90629,1,1,1,N,117,7,1,9.5,2.15,1,12.95
3,90769,1,90770,1,1,1,N,89066,7260,1,10.0,1.50,1,12.80
4,64705,2,64706,1,1,1,N,63602,63800,1,9.5,1.00,1,11.80
